In [1]:
using GAMS

In [2]:
using JuMP

In [3]:
using GLPK

In [4]:
using DataStructures

In [5]:
using Surrogates

In [6]:
using Flux

In [7]:
using DataFrames

# Novel mathematical formulation for the short-term scheduling of batch plants: Example 2

This GAMS Model presents an application of the novel mathematical formulation for the short-term scheduling of batch plants. The proposed formulation by Ierapetritou and Floudas  is used on a literature example by Sundaramoorthy and Karimi that involves 3 tasks and 5 units (2 mixers, 1 reactor and 2 purificators). The main process involves a series of  processing tasks i.e. a mixing, reaction and purification.

Ierapetritou and Floudas (1998) Effective Continuous-Time Formulation for Short-Term Scheduling. 1. Multipurpose Batch Processes. Independent Engineering Chemical Research 37 (4341-4359)

Kondili E, Pantelides CC, Sargent RWH. (1993) A general algorithm for Short-term scheduling of batch operations. I. MILP formulation. Computer Chemical Engineering. 17 (211–227).

In [8]:
H = 50;
P = 16;

# SETS
Define the sets to be utilised in the MILP
Please task note that if a task can be performed in more than one unit then it is considered a different task for each unit it can be performed in.

In [9]:
#Sets

I = ["t1","t2","t3","t4","t5"];
J = ["j1","j2","j3","j4","j5"];
N = range(1,P, step = 1);
S = ["s1","s2","s3","s4"];

Jt1 = [J[1]];
Jt2 = [J[2]];
Jt3 = [J[3]];
Jt4 = [J[4]];
Jt5 = [J[5]];

Ij1 = [I[1]];
Ij2 = [I[2]];
Ij3 = [I[3]];
Ij4 = [I[4]];
Ij5 = [I[5]];

Is1 = I[1:2]
Is2 = I[1:3]
Is3 = I[3:5]
Is4 = I[4:5]

Ip = I;
Jp = J;
Np = N;


# PARAMETERS
The available information for the process is given below i.e. storage capacity, initial available inventory,
batch size and parameters alpha and beta which are the processing time parameters viz. constant and variable terms


In [10]:
#PARAMETERS
#bmin = [0,0,0,0,0]

vmin = OrderedDict(
   "t1" => 0,
   "t2" => 0,
   "t3" => 0,
   "t4" => 0,
   "t5" => 0   
)

#bmax = [100,150,200,150,150]
vmax = OrderedDict(
   "t1" => 100,
   "t2" => 150,
   "t3" => 200,
   "t4" => 150,
   "t5" => 150 
)

#STin = [10000, 0, 0, 0]
STin = OrderedDict(
     "s1" => 10000,
     "s2" => 0,
     "s3" => 0,
     "s4" => 0)

#STmax = [+inf, 200, 250, +inf ]
STmax = OrderedDict(
     "s1" => 100000,
     "s2" => 200,
     "s3" => 250,
     "s4" => 0
)

#alpha = [1.333, 1.333, 1.000, 0.667, 0.667]
alpha = OrderedDict(
"t1" => 1.333,
"t2" => 1.333,
"t3" => 1.000,
"t4" => 0.667,
"t5" => 0.667
)

#beta = [0.01333, 0.01333, 0.00500, 0.00445, 0.00445]
beta = OrderedDict(
"t1" => 0.01333,
"t2" => 0.01333,
"t3" => 0.00500,
"t4" => 0.00445,
"t5" => 0.00445       
)

price = OrderedDict(
"s1" => 0,
"s2" => 0,
"s3" => 0,
"s4" => 5
)

demand = OrderedDict(
"s1" => 0,
"s2" => 0,
"s3" => 0,
"s4" => 2000
)

#rho_table = wsv"""
#i        s1       s2     s3    s4
#t1       -1       +1      0     0
#t2       -1       +1      0     0
#t3        0       -1     +1     0
#t4        0        0     -1    +1
#t5        0        0     -1    +1
#"""

rho_table = DataFrame([(I = "t1", s1 = -1, s2 = +1, s3 = 0, s4 = 0 ),
                   (I = "t2", s1 = -1, s2 = +1, s3 = 0, s4 = 0 ),
                   (I = "t3", s1 = 0, s2 = -1, s3 = +1, s4 = 0 ),
                   (I = "t4", s1 = 0, s2 = 0, s3 = -1, s4 = +1 ),
                   (I = "t5", s1 = 0, s2 = 0, s3 = -1, s4 = +1 )]);

rho = OrderedDict( (r[:I],states) => r[Symbol(states)] for r in eachrow(rho_table), states in S);

# MODEL DEFINITION

In [11]:
example2_makespan = Model(GLPK.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK

# VARIABLES

In [12]:
@variables example2_makespan begin
    w[i in I, n in N], Bin
    y[j in J, n in N], Bin
    bm[i in I, j in J, n in N] >= 0
    d[s in S, n in N] >= 0
    ST[s in S, n in N] >= 0
    Ts[i in I, j in J, n in N] >= 0
    Tf[i in I, j in J, n in N] >= 0
    MS >= 0


end

# ALLOCATION CONSTRAINTS
The allocation constraints express that at each unit j and at an event point n only one of the tasks that can be performed
in this unit (i.e. i ∈ Ij ) should take place

In [13]:
@constraints example2_makespan begin
    allocation1[j in J, n in N; j == J[1]],
      sum(w[i,n] for i in Ij1) == y[j,n]
    allocation2[j in J, n in N; j == J[2]],
      sum(w[i,n] for i in Ij2) == y[j,n]  
    allocation3[j in J, n in N; j == J[3]],
      sum(w[i,n] for i in Ij3) == y[j,n]
    allocation4[j in J, n in N; j == J[4]],
      sum(w[i,n] for i in Ij4) == y[j,n]  
    allocation5[j in J, n in N; j == J[5]],
      sum(w[i,n] for i in Ij5) == y[j,n]
end

# CAPACITY CONSTRAINTS

The capacity constraints specify the minimum or maximum amount of available material for a task to take place in a unit
Since vmin is zero for all of them, the minimum capacity constraint is not defined

In [14]:
@constraints example2_makespan begin
    capacity1[i in I, j in Jt1, n in N],
      bm[i,j,n] <= vmax[i]*w[i,n]
    capacity2[i in I, j in Jt2, n in N],
      bm[i,j,n] <= vmax[i]*w[i,n]
    capacity3[i in I, j in Jt3, n in N],
      bm[i,j,n] <= vmax[i]*w[i,n]
    capacity4[i in I, j in Jt4, n in N],
      bm[i,j,n] <= vmax[i]*w[i,n]
    capacity5[i in I, j in Jt5, n in N],
      bm[i,j,n] <= vmax[i]*w[i,n]

end

# STORAGE CONSTRAINTS
This constraint specifies the maximum storage capacity of each material state

In [15]:
@constraints example2_makespan begin
    storage[s in S, n in N],
      ST[s,n] <= STmax[s]
end

# MATERIAL BALANCE: INITIAL CONDITIONS
This first set constraints is specifying the initial amount of each material state at event point 1


In [16]:
@constraints example2_makespan begin
    matbal1a[s in S, n in N; s == S[1] && n == N[1]],
      ST[s,n] == STin[s] + sum(rho[i,s]*bm[i,j,n] for i in Is1 for j in Jt1) + sum(rho[i,s]*bm[i,j,n] for i in Is1 for j in Jt2) - d[s,n]
    matbal1b[s in S, n in N; s == S[2] && n == N[1]],
      ST[s,n] == STin[s] + sum(rho[i,s]*bm[i,j,n] for i in Is2 for j in Jt3) - d[s,n]
    matbal1c[s in S, n in N; s == S[3] && n == N[1]],
      ST[s,n] == STin[s] + sum(rho[i,s]*bm[i,j,n] for i in Is3 for j in Jt4) + sum(rho[i,s]*bm[i,j,n] for i in Is3 for j in Jt5) - d[s,n]
    matbal1d[s in S, n in N; s == S[4] && n == N[1]],
      ST[s,n] == STin[s] - d[s,n]
    
end

# MATERIAL BALANCE PROCESSED
The second set of material balance performs a mass balnce on each state i.e. the amount of material s consumed at event point n by task i in unit j and the amount of material s produced at event point n-1 by task i in unit j


In [17]:
@constraints example2_makespan begin
    matbal2a[s in S, n in 2:length(N); s == S[1]],
      ST[s,n] == ST[s,n-1] + sum(rho[i,s]*bm[i,j,n] for j in Jt1 for i in Is1) + sum(rho[i,s]*bm[i,j,n] for j in Jt2 for i in Is1) - d[s,n]
    matbal2b[s in S, n in 2:length(N); s == S[2]],
      ST[s,n] == ST[s,n-1] + sum(rho[i,s]*bm[i,j,n-1] for j in Jt1 for i in Is2) + sum(rho[i,s]*bm[i,j,n-1] for j in Jt2 for i in Is2 ) + sum(rho[i,s]*bm[i,j,n] for j in Jt3 for i in Is2 ) - d[s,n]
    matbal2c[s in S, n in 2:length(N); s == S[3]],
      ST[s,n] == ST[s,n-1] + sum(rho[i,s]*bm[i,j,n-1] for j in Jt3 for i in Is3) + sum(rho[i,s]*bm[i,j,n] for j in Jt4 for i in Is3 ) + sum(rho[i,s]*bm[i,j,n] for j in Jt5 for i in Is3 ) - d[s,n]
    matbal2d[s in S, n in 2:length(N); s == S[4]],
      ST[s,n] == ST[s,n-1] + sum(rho[i,s]*bm[i,j,n-1] for j in Jt4 for i in Is4 ) + sum(rho[i,s]*bm[i,j,n-1] for j in Jt5 for i in Is4 ) - d[s,n]
    
end

# DEMAND CONSTRAINTS

In [18]:
@constraints example2_makespan begin
    demand[s in S],
      sum(d[s,n] for n in N if n == length(N)) >= demand[s]
    
end

# DURATION CONSTRAINTS
These constraints don't only specify how long task i will take in unit j but also specifies the dependence of the duration
on the amount of material to be processed by task i in unit j.

In [19]:
@constraints example2_makespan begin
    duration1[i in I, j in Jt1, n in N],
      Tf[i,j,n] == Ts[i,j,n] + alpha[i]*w[i,n] + beta[i]*bm[i,j,n]
    duration2[i in I, j in Jt2, n in N],
      Tf[i,j,n] == Ts[i,j,n] + alpha[i]*w[i,n] + beta[i]*bm[i,j,n]
    duration3[i in I, j in Jt3, n in N],
      Tf[i,j,n] == Ts[i,j,n] + alpha[i]*w[i,n] + beta[i]*bm[i,j,n]
    duration4[i in I, j in Jt4, n in N],
      Tf[i,j,n] == Ts[i,j,n] + alpha[i]*w[i,n] + beta[i]*bm[i,j,n]
    duration5[i in I, j in Jt5, n in N],
      Tf[i,j,n] == Ts[i,j,n] + alpha[i]*w[i,n] + beta[i]*bm[i,j,n]

end

# SEQUENCE CONSTRAINTS

# i. Same Task in Same Unit
The first set of sequence specify that the start of task i at event point n+1 should start after the end of event point n for the same task performed in unit j.


In [20]:
@constraints example2_makespan begin
    sequence1a[i in I, j in Jt1, n in 1:length(N)-1],
      Ts[i,j,n+1] >= Tf[i,j,n] -  H*(2 - w[i,n]- y[j,n])
    sequence1b[i in I, j in Jt2, n in 1:length(N)-1],
      Ts[i,j,n+1] >= Tf[i,j,n] - H*(2 - w[i,n]- y[j,n])
    sequence1c[i in I, j in Jt3, n in 1:length(N)-1],
      Ts[i,j,n+1] >= Tf[i,j,n] - H*(2 - w[i,n]- y[j,n])
    sequence1d[i in I, j in Jt4, n in 1:length(N)-1],
      Ts[i,j,n+1] >= Tf[i,j,n] - H*(2 - w[i,n]- y[j,n])
    sequence1e[i in I, j in Jt5, n in 1:length(N)-1],
      Ts[i,j,n+1] >= Tf[i,j,n] - H*(2 - w[i,n]- y[j,n])

    sequence2a[i in I, j in Jt1, n in 1:length(N)-1],
      Ts[i,j,n+1] >= Ts[i,j,n]
    sequence2b[i in I, j in Jt2, n in 1:length(N)-1],
      Ts[i,j,n+1] >= Ts[i,j,n]
    sequence2c[i in I, j in Jt3, n in 1:length(N)-1],
      Ts[i,j,n+1] >= Ts[i,j,n]
    sequence2d[i in I, j in Jt4, n in 1:length(N)-1],
      Ts[i,j,n+1] >= Ts[i,j,n]
    sequence2e[i in I, j in Jt5, n in 1:length(N)-1],
      Ts[i,j,n+1] >= Ts[i,j,n]

    sequence3a[i in I, j in Jt1, n in 1:length(N)-1],
      Tf[i,j,n+1] >= Tf[i,j,n]
    sequence3b[i in I, j in Jt2, n in 1:length(N)-1],
      Tf[i,j,n+1] >= Tf[i,j,n]
    sequence3c[i in I, j in Jt3, n in 1:length(N)-1],
      Tf[i,j,n+1] >= Tf[i,j,n]
    sequence3d[i in I, j in Jt4, n in 1:length(N)-1],
      Tf[i,j,n+1] >= Tf[i,j,n]
    sequence3e[i in I, j in Jt5, n in 1:length(N)-1],
      Tf[i,j,n+1] >= Tf[i,j,n]

end

# ii. Different Task in Same Unit
The following constraints establishes the relationship between the starting time of task i at point n+1 and the end time of task i′ (ip) at event point n when different tasks take place in the same unit.

In [21]:
@constraints example2_makespan begin
    sequence4a[i in Ij1, ip in Ij1, j in J, n in 1:length(N)-1; i != ip],
      Ts[i,j,n+1] >= Tf[ip,j,n] - H*(2 - w[ip,n] - y[j,n])
    sequence4b[i in Ij2, ip in Ij2, j in J, n in 1:length(N)-1; i != ip],
      Ts[i,j,n+1] >= Tf[ip,j,n] - H*(2 - w[ip,n] - y[j,n])
    sequence4c[i in Ij3, ip in Ij3, j in J, n in 1:length(N)-1; i != ip],
      Ts[i,j,n+1] >= Tf[ip,j,n] - H*(2 - w[ip,n] - y[j,n])
    sequence4d[i in Ij2, ip in Ij4, j in J, n in 1:length(N)-1; i != ip],
      Ts[i,j,n+1] >= Tf[ip,j,n] - H*(2 - w[ip,n] - y[j,n])
    sequence4e[i in Ij3, ip in Ij5, j in J, n in 1:length(N)-1; i != ip],
      Ts[i,j,n+1] >= Tf[ip,j,n] - H*(2 - w[ip,n] - y[j,n])
   
end

# ii. Different Task in Different Units

When different tasks i and i′ are performed in different units j and j′ but take place one after the other according to the production recipe.These constraints specify the order in which then tasks in each unit should be performed i.e. mixing then reaction then purification.


In [22]:
@constraints example2_makespan begin
    sequence5a[i in I, ip in I, j in Jt3, jp in J, n in 1:length(N)-1; i == I[3] && ip == I[1] && jp == J[1]],
      Ts[i,j,n+1] >= Tf[ip,jp,n] - H*(2 - w[ip,n] - y[jp,n])
    sequence5b[i in I, ip in I, j in Jt3, jp in J, n in 1:length(N)-1; i == I[3] && ip == I[2] && jp == J[2]],
      Ts[i,j,n+1] >= Tf[ip,jp,n] - H*(2 - w[ip,n] - y[jp,n])
    sequence5c[i in I, ip in I, j in Jt4, jp in J, n in 1:length(N)-1; i == I[4] && ip == I[3] && jp == J[3]],
      Ts[i,j,n+1] >= Tf[ip,jp,n] - H*(2 - w[ip,n] - y[jp,n])
    sequence5d[i in I, ip in I, j in Jt5, jp in J, n in 1:length(N)-1; i == I[5] && ip == I[3] && jp == J[3]],
      Ts[i,j,n+1] >= Tf[ip,jp,n] - H*(2 - w[ip,n] - y[jp,n])
    sequence5e[i in I, ip in I, j in Jt3, jp in J, n in 1:length(N)-1; i == I[3] && ip == I[4] && jp == J[4]],
      Ts[i,j,n+1] >= Tf[ip,jp,n] - H*(2 - w[ip,n] - y[jp,n])
    sequence5f[i in I, ip in I, j in Jt3, jp in J, n in 1:length(N)-1; i == I[3] && ip == I[5] && jp == J[5]],
      Ts[i,j,n+1] >= Tf[ip,jp,n] - H*(2 - w[ip,n] - y[jp,n])
    
end

# iv. Completion of previous tasks
A task i' (ip) performed in unit j cannot start until task i in unit j is completed


In [23]:
@constraints example2_makespan begin
   sequence6a[i in I, j in Jt1, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij1)
   sequence6b[i in I, j in Jt1, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij2)
   sequence6c[i in I, j in Jt1, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij3)
   sequence6d[i in I, j in Jt1, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij4)
   sequence6e[i in I, j in Jt1, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij5)

   sequence7a[i in I, j in Jt2, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij1)
   sequence7b[i in I, j in Jt2, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij2)
   sequence7c[i in I, j in Jt2, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij3)
   sequence7d[i in I, j in Jt2, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij4)
   sequence7e[i in I, j in Jt2, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij5)

   sequence8a[i in I, j in Jt3, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij1)
   sequence8b[i in I, j in Jt3, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij2)
   sequence8c[i in I, j in Jt3, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij3)
   sequence8d[i in I, j in Jt3, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij4)
   sequence8e[i in I, j in Jt3, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij5)

   sequence9a[i in I, j in Jt4, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij1)
   sequence9b[i in I, j in Jt4, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij2)
   sequence9c[i in I, j in Jt4, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij3)
   sequence9d[i in I, j in Jt4, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij4)
   sequence9e[i in I, j in Jt4, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij5)

   sequence10a[i in I, j in Jt5, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij1)
   sequence10b[i in I, j in Jt5, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij2)
   sequence10c[i in I, j in Jt5, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij3)
   sequence10d[i in I, j in Jt5, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij4)
   sequence10e[i in I, j in Jt5, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij5)

end

# TIME HORIZON CONSTRAINTS
Specify that all tasks should task place within the time horizon


In [24]:
#TIME HORIZON CONSTRAINTS
@constraints example2_makespan begin
    timehorizon1[i in I, j in J, n in N],
      Ts[i,j,n] <= H
    timehorizon2[i in I, j in J, n in N],
      Tf[i,j,n] <= H
    
end

# OBJECTIVE FUNCTION

## Makespan Minimization

In [25]:
@constraints example2_makespan begin
    makespan[i in I, j in J, n in N; n == length(N)],
       Tf[i,j,n] <= MS    
end

In [26]:
@objective example2_makespan Min begin
    MS
end

MS

In [27]:
example2_makespan

A JuMP Model
Minimization problem with:
Variables: 1489
Objective function type: VariableRef
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 544 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 3244 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 1289 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 1329 constraints
`VariableRef`-in-`MathOptInterface.ZeroOne`: 160 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK
Names registered in the model: MS, ST, Tf, Ts, allocation1, allocation2, allocation3, allocation4, allocation5, bm, capacity1, capacity2, capacity3, capacity4, capacity5, d, demand, duration1, duration2, duration3, duration4, duration5, makespan, matbal1a, matbal1b, matbal1c, matbal1d, matbal2a, matbal2b, matbal2c, matbal2d, sequence10a, sequence10b, sequence10c, sequence10d, sequence10e, sequence1a

In [28]:
optimize!(example2_makespan)

# Optimal value and Schedule

In [29]:
println("Optimal Solution: ", JuMP.objective_value(example2_makespan))
println("Optimal Schedule: ")
println(JuMP.value.(w))

LoadError: [91mResult index of attribute MathOptInterface.ObjectiveValue(1) out of bounds. There are currently 0 solution(s) in the model.[39m